In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

In [2]:
# Functions
nn_p = 3
nn_w = 5
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
# nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+nn_p(\d+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
# this line and nn_decode_keys will change for nn_Norm
nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)nn_w(\d+)nn_p(\d+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','nnw','nnp','div','method','replace_mech']

def get_files(m): return list(path.glob(f'**/nonn/**/*{m}.csv'))
def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))

def read_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(decode_keys,pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'] + '_' + df['div']
#     df['method'] = df['method'].str.replace('noNNRestart', 'noNN_Rst')
    df['method'] = df['method'].str.replace('noNNRestart_No', 'noNN_Rst')
    df.drop('div', axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'] + '_' + df['replace_mech'] + '_' + df['div']
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    files = get_files(m)
    nn_files = get_nn_files(m)
    data = pd.concat([read_csv(f,m) for f in files] + [read_nn_csv(f,m) for f in nn_files])
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data.reset_index(drop=True)
    return data

def plot_one(data, m, normalize=False, title='', title_size=14, col_by_freq=col_by_freq, col_palette=col_palette,
             legend=False, hide_x=True, hide_y=True, ax=None, do_lim=True, ll=0.2, ul=0.8):
    m = m.upper()
    if normalize: m = m + '_norm' 
    if ax is None: fig,ax = plt.subplots(1,1,figsize=(6,4))
    sns.boxplot('method' if col_by_freq else 'freq', m, hue='freq' if col_by_freq else 'method',
                data=data, palette=col_palette,
                # width=2.5,
                linewidth=0.5, fliersize=0, ax=ax)
    if not legend: ax.get_legend().remove()
    if do_lim:
        ax.set_ylim(-0.05,1.05)
    else:
        g = data.groupby(['method','freq'])[m]
        q1,q3 = g.quantile(ll),g.quantile(ul)
        iqr = q3-q1
        lower_lim = (q1 - 1.5*iqr).min()
        upper_lim = (q3 + 1.5*iqr).max()
        lower_lim = data[m][data[m]>=lower_lim].min()
        upper_lim = data[m][data[m]<=upper_lim].max()
        ax.set_ylim(lower_lim,upper_lim)
        
    ax.set_xlabel('')
    if hide_x: ax.set_xticklabels([])
    if hide_y: ax.set_ylabel('')
    ax.set_title(title, size=title_size)
    return ax

def plot_all_vertical(m, normalize=False, title_size=14, col_by_freq=col_by_freq, col_palette=col_palette, do_lim=True,
                      ll=0.2, ul=0.8):
    data = get_data(m, normalize=normalize)
#     fig,axss = plt.subplots(4, 3, figsize=(14,16), sharex=do_lim, sharey=do_lim)
    fig,axss = plt.subplots(4, 3, figsize=(26,18), sharex=False, sharey=False)
    for i,(axs,exp) in enumerate(zip(axss,['exp1','exp2','exp3','exp4'])):
        first_exp = i==0
        for j,(ax,func) in enumerate(zip(axs,['sphere', 'rosenbrock', 'rastrigin'])):
            first_func = j==0
            test = data.query(f'experiment=={exp!r} and function=={func!r}').sort_values('method', ascending=False)
            plot_one(test, m, normalize=normalize, title=f'{exp}-{func.title()}', title_size=title_size, hide_x=False,
                     hide_y=not first_func, col_by_freq=col_by_freq, col_palette=col_palette, ax=ax,
                     do_lim=do_lim, ll=ll, ul=ul)
        ax.legend(loc='upper left', ncol=3)
    plt.tight_layout()
    return fig,axss

def plot_all_horizontal(m, normalize=False, title_size=14, col_by_freq=col_by_freq, col_palette=col_palette, do_lim=True,
                        ll=0.2, ul=0.8):
    data = get_data(m, normalize=normalize)
#     fig,axss = plt.subplots(4, 3, figsize=(14,16), sharex=do_lim, sharey=do_lim)
    fig,axss = plt.subplots(3, 4, figsize=(28,20), sharex=False, sharey=False)#20,12
    for i,exp in enumerate(['exp1','exp2','exp3','exp4']):#
        first_exp = i==0
        axs = axss[:,i]
        for j,(ax,func) in enumerate(zip(axs,['sphere', 'rosenbrock', 'rastrigin'])):
            first_func = i==0
            test = data.query(f'experiment=={exp!r} and function=={func!r}').sort_values('method', ascending=False)
            plot_one(test, m, normalize=normalize, title=f'{exp}-{func.title()}', title_size=title_size, hide_x=False,
                     hide_y=not first_func, col_by_freq=col_by_freq, col_palette=col_palette, ax=ax,
                     do_lim=do_lim, ll=ll, ul=ul)
        
    for axs in axss: axs[-1].legend(loc='upper right', ncol=3)
    plt.tight_layout()
    return fig,axss

# Plots

In [3]:
labels_order = ['noNN_RI', 'NN_RI','noNN_HMu',  'NN_HMu', 'noNN_CwN', 'NN_CwN', 'noNN_No', 
                 'NN_No','noNN_Rst']

In [24]:
def get_heatmap_data(m):
    df = get_data(m)
    df = df[df.nnp.isna() | (df.nnp == str(nn_p))].drop('nnp', axis=1)
    df.function = df.function.str.title()
    m = m.upper()
    df_pivot = df.pivot_table(index=['experiment','function','freq'], columns=['method'], values=[m])[m]
    return df_pivot

In [25]:
df_pivot = get_heatmap_data('mof')#[labels_order]
df_pivot_rank = df_pivot.rank(axis=1)
df_pivot = df_pivot.reset_index()
df_pivot.experiment = df_pivot.experiment.str.slice(3)
df_pivot.set_index(['experiment','function','freq'], inplace=True)

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
df_pivot = get_heatmap_data('mof')[labels_order]
df_pivot_rank = df_pivot.rank(axis=1)
df_pivot = df_pivot.reset_index()
df_pivot.experiment = df_pivot.experiment.str.slice(3)
df_pivot.set_index(['experiment','function','freq'], inplace=True)

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [28]:
# df_pivot = df_pivot.query('freq==10')
df_pivot

method                            NN_CwN        NN_HMu         NN_No  \
experiment function   freq                                             
1          Rastrigin  0.5   1.235115e+01  6.316635e+00  1.659610e+01   
                      1.0   1.365771e+01  4.989107e+00  1.644207e+01   
                      4.0   1.076442e+01  2.867563e+00  1.587269e+01   
                      10.0  7.176945e+00  1.943430e+00  1.549495e+01   
                      20.0  5.645420e+00  1.495584e+00  1.548770e+01   
           Rosenbrock 0.5   2.314646e+04  5.338951e+03  6.399246e+03   
                      1.0   1.985397e+04  3.302664e+03  4.484306e+03   
                      4.0   1.542188e+04  1.294024e+03  2.692885e+03   
                      10.0  1.371349e+04  6.680167e+02  2.119930e+03   
                      20.0  1.226057e+04  5.407704e+02  1.840048e+03   
           Sphere     0.5   6.814421e+00  1.623836e+00  2.138037e+00   
                      1.0   5.661169e+00  9.925341e-01  1.414361e+00   
                      4.0   4.108776e+00  3.914149e-01  8.049631e-01   
                      10.0  3.459034e+00  2.214940e-01  6.346443e-01   
                      20.0  3.044791e+00  1.437996e-01  5.632986e-01   
2          Rastrigin  0.5   5.359248e+00  5.895830e+00  3.262107e+00   
                      1.0   4.218468e+00  4.367956e+00  1.836143e+00   
                      4.0   2.348547e+00  1.902284e+00  1.083127e+00   
                      10.0  1.812777e+00  1.137487e+00  1.054395e+00   
                      20.0  1.548690e+00  1.009712e+00  1.040332e+00   
           Rosenbrock 0.5   6.301014e+02  4.176209e+02  1.232253e+02   
                      1.0   4.469396e+02  2.287541e+02  5.352661e+01   
                      4.0   2.328139e+02  6.819022e+01  1.403388e+01   
                      10.0  1.474375e+02  3.437630e+01  5.975013e+00   
                      20.0  1.104352e+02  1.887414e+01  3.225551e+00   
           Sphere     0.5   7.413697e-01  4.678375e-01  1.590142e-01   
                      1.0   5.909773e-01  2.490606e-01  7.596898e-02   
                      4.0   3.729349e-01  7.793368e-02  2.260975e-02   
                      10.0  2.785955e-01  3.511824e-02  9.591511e-03   
                      20.0  2.237719e-01  1.978196e-02  5.080683e-03   
3          Rastrigin  0.5   1.097683e+01  2.312199e+00  8.028306e+00   
                      1.0   1.073977e+01  2.118492e+00  1.540714e+01   
                      4.0   1.195357e+01  2.815107e+00  2.899243e+01   
                      10.0  1.072366e+01  3.448223e+00  3.532047e+01   
                      20.0  1.190721e+01  3.614509e+00  3.799971e+01   
           Rosenbrock 0.5   5.363213e+02  7.400487e+01  6.169929e+01   
                      1.0   3.200752e+02  2.997970e+01  2.499669e+01   
                      4.0   1.636538e+02  6.510480e+00  6.066935e+00   
                      10.0  9.304933e+01  2.898612e+00  2.666000e+00   
                      20.0  6.555379e+01  1.710130e+00  1.555736e+00   
           Sphere     0.5   6.644446e-01  7.115898e-02  5.814244e-02   
                      1.0   4.742167e-01  3.166061e-02  2.660453e-02   
                      4.0   2.762370e-01  7.439737e-03  6.799829e-03   
                      10.0  2.039599e-01  7.812903e-03  4.984198e-03   
                      20.0  1.657636e-01  1.119958e-02  7.819098e-03   
4          Rastrigin  0.5   1.295419e+02  5.375092e+01  7.779314e+01   
                      1.0   1.139200e+02  2.841952e+01  5.257841e+01   
                      4.0   8.094219e+01  8.707500e+00  2.782591e+01   
                      10.0  5.837510e+01  4.903019e+00  2.558427e+01   
                      20.0  4.336513e+01  3.342344e+00  2.400716e+01   
           Rosenbrock 0.5   3.958816e+07  1.049785e+07  1.614997e+07   
                      1.0   3.428678e+07  4.174695e+06  7.442566e+06   
                      4.0   2.268749e+07  1.032453e+06  6.759168e+06   
                      10.0  1.525531e+07

In [29]:
df_pivot.style.background_gradient(cmap=plt.cm.Greens, axis=1)

invalid value encountered in less


In [33]:
df_pivot_rank.style.background_gradient(cmap=plt.cm.Greens, axis=1)

In [34]:
summary = df_pivot.groupby(['experiment', 'function']).mean()
summary.round(2).style.background_gradient(cmap=plt.cm.Greens, axis=1)

In [35]:
from ipywidgets import interact

def show_table(dm):
    out = summary[[f'noNN_{dm}', f'NN_{dm}']].round(2)
    return out.style.background_gradient(cmap=plt.cm.Blues_r, axis=1)
    
interact(show_table, dm=['RI', 'HMu','No'])

interactive(children=(Dropdown(description='dm', options=('RI', 'HMu', 'No'), value='RI'), Output()), _dom_cla…

<function __main__.show_table(dm)>

In [36]:
out = summary.round(2).style
for i in range(len(labels_order)//2): 
    out = out.highlight_min(axis=1, subset=labels_order[i*2:(i+1)*2])

out

In [37]:
[meth for meth in [labels_order]]

[['noNN_RI',
  'NN_RI',
  'noNN_HMu',
  'NN_HMu',
  'noNN_CwN',
  'NN_CwN',
  'noNN_No',
  'NN_No',
  'noNN_Rst']]

In [96]:
Doc(Styler)

NameError: name 'Doc' is not defined

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(summary, annot=True, linewidths=.5, ax=ax, cmap=sns.cm.rocket, vmin=-1)
u,l = ax.get_ylim()
ax.set_ylim(u+0.5, l-0.5)
ax.set_ylabel('')
ax.set_xlabel('')
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(12)
    tick.label.set_rotation(40)
    
for tick in ax.yaxis.get_major_ticks(): tick.label.set_fontsize(12)
plt.tight_layout()
# fig.savefig(path/'heatmapMOF.eps', dpi=400, format='eps')